In [1]:
import imp
model = imp.load_source('model', '../modules/model.py')

In [2]:
tf_graph = model.getGraph()
session = model.init_interactive_session(tf_graph)

In [3]:
model.load_saved_model('/flat2/saved_models/SVHN_3_digit_May7', session)